In [1]:
#Modules
import pandas as pd
import numpy as np
import warnings
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
warnings.filterwarnings("ignore")

In [2]:
# Load data
df = pd.read_csv("Market Prices Search.csv")

print("Raw data shape:", df.shape)

# View first rows
print(df.head())


             Market  Commodity Classification Grade Sex Wholesale     Retail  \
0           Gikomba  Dry Maize   Yellow Maize     -   -  77.78/Kg  100.00/Kg   
1           Gikomba  Dry Maize    White Maize     -   -  50.00/Kg   60.00/Kg   
2          Kerugoya  Dry Maize    White Maize     -   -  40.00/Kg   65.00/Kg   
3  Ngurubani Market  Dry Maize    White Maize     -   -  46.67/Kg   50.00/Kg   
4           Gikomba  Dry Maize   Yellow Maize     -   -  77.78/Kg  100.00/Kg   

   Supply Volume     County        Date  
0            NaN    Nairobi  2025-12-16  
1            NaN    Nairobi  2025-12-16  
2         4500.0  Kirinyaga  2025-12-15  
3         5450.0  Kirinyaga  2025-12-15  
4            NaN    Nairobi  2025-12-15  


In [4]:
# keep only dry maize
df = df[df["Commodity"].str.lower().str.contains("maize")]


In [5]:
# convert price to numeric
df["Retail"] = pd.to_numeric(
    df["Retail"]
    .str.replace("/Kg", "", regex=False),
    errors="coerce"
)

In [6]:
# convert date to datetime
df["Date"] = pd.to_datetime(df["Date"])

In [7]:
df.describe()
print("After cleaning:", df.shape)

,Retail,Supply Volume,Date
count,2880.000000,1281.000000,3000
mean,79.608337,5553.990632,2024-03-13 00:45:35.999999744
min,6.390000,0.000000,2022-12-13 00:00:00
25%,60.000000,3000.000000,2023-06-05 00:00:00
50%,70.000000,5000.000000,2024-01-24 00:00:00
75%,90.000000,6300.000000,2024-09-10 06:00:00
max,7000.000000,80000.000000,2025-12-16 00:00:00
std,163.749751,5518.131324,NaN


In [8]:
# Convert Daily Prices to Weekly Prices
weekly = (
    df.groupby(
        ["County", pd.Grouper(key="Date", freq="W-MON")]
    )["Retail"]
    .mean()
    .reset_index()
    .rename(columns={"Retail": "price"})
)

print(weekly.head())


   County       Date  price
0  Kiambu 2023-01-23   85.0
1  Kiambu 2023-02-06   85.0
2  Kiambu 2023-02-20   90.0
3  Kiambu 2023-06-05   95.0
4  Kiambu 2023-06-19   95.0


In [9]:
# lag features
for lag in [1, 2, 3, 4]:
    weekly[f"lag_{lag}"] = (
        weekly.groupby("County")["price"].shift(lag)
    )


In [10]:
# Rolling statistics
weekly["roll_mean_4"] = (
    weekly.groupby("County")["price"]
    .rolling(4).mean()
    .reset_index(level=0, drop=True)
)

weekly["roll_std_4"] = (
    weekly.groupby("County")["price"]
    .rolling(4).std()
    .reset_index(level=0, drop=True)
)


In [11]:
# Calender features
weekly["week"] = weekly["Date"].dt.isocalendar().week
weekly["month"] = weekly["Date"].dt.month
weekly["year"] = weekly["Date"].dt.year


In [12]:
# Encode counties
weekly = pd.get_dummies(weekly, columns=["County"])

# Remove missing rows from lagging
weekly = weekly.dropna().reset_index(drop=True)


In [14]:
# Define features(X) and target(Y)
X = weekly.drop(columns=["price", "Date"])
y = weekly["price"]
